In [2]:
import pandas as pd
import random

df_Videos = pd.read_csv('Videos.csv')
df_PersonVideo = pd.read_csv('PersonVideo.csv')

df = pd.merge(df_Videos,df_PersonVideo, left_on='IdVideo', right_on='IdVideo')
df = df.set_index('IdVideo')

valid_id_list = df.index.to_list()
idlist = [random.choice(valid_id_list) for i in range(20)]

idlist.append(425) # this one has the person partially enter the picture before the given frame
idlist.append(286) # this one had incorrect path formatting in db
idlist.append(294) # this one has filenames formatted differently

In [4]:
import cv2
from pathlib import Path, PureWindowsPath
import os
import re

for idvideo in idlist:
    framestart = df.FrameStart[idvideo]
    left = df.BBLeft[idvideo]
    top = df.BBTop[idvideo]
    width = df.Width[idvideo]
    height = df.Height[idvideo]

    # db contains windows paths, convert to linux
    path = df.FilePathRelative[idvideo]
    path = PureWindowsPath(path)
    path = str(Path(path))
    # now find the frame with the right id, accounting for varying file names
    filenames = os.listdir(path)
    p = re.compile('.*' + '{:04d}'.format(framestart) + '\..*')
    selected_file = list(filter(p.search, filenames))
    assert len(selected_file) is 1 # confirm there's no errors
    path = os.path.join(path, selected_file[0])
    

    im = cv2.imread(path)
    cv2.imwrite('bb_tests/{}_raw.jpg'.format(idvideo),im)
    cv2.rectangle(im,(left,top),(left+width,top + height),(0,255,0),2)
    cv2.imwrite('bb_tests/{}_bb.jpg'.format(idvideo),im)